In [33]:
# Start by importing necessary packages
import requests
import pandas as pd
import numpy as np
from IPython.display import display
from io import StringIO

from dstapi import DstApi # The helper class

In [34]:
# Directly embed parameters in the URL with response.get()
requests.get('https://api.statbank.dk/v1' + '/tableinfo' + "?id=metrox1&format=JSON").json()

# Pass a dictionary of parameters to requests.get()
params = {'id': 'NABK69', 'format': 'JSON'}
requests.get('https://api.statbank.dk/v1' + '/tableinfo', params=params).json()

# Use response.post() - note the change in the name of the parameter about the table's name
# I'm also adding here a language parameter - most tables are available in both Danish and English
params = {'table': 'NABK69', 'format': 'JSON', 'lang':'da'}
requests.post('https://api.statbank.dk/v1' + '/tableinfo', json=params).json()

{'id': 'NABK69',
 'text': 'Akkumulations- og statuskonti, investering og beholdning af faste aktiver (69-gruppering)',
 'description': 'Akkumulations- og statuskonti, investering og beholdning af faste aktiver (69-gruppering) efter beholdning / strøm, aktiv, branche, prisenhed og tid',
 'unit': 'Mio. kr.',
 'suppressedDataValue': '0',
 'updated': '2025-07-04T08:00:00',
 'active': True,
 'contacts': [{'name': 'Bo Siemsen',
   'phone': '+4521579724',
   'mail': 'bsm@dst.dk'}],
 'documentation': {'id': 'caf4e327-d063-497a-94da-1997828c77f8',
  'url': 'https://www.dst.dk/statistikdokumentation/caf4e327-d063-497a-94da-1997828c77f8'},
 'footnote': {'text': 'Vi har ekstraordinært opdateret denne tabel 4/7-2025. Det er desværre konstateret at vi har offentliggjort en dobbeltregning af udvalgte værdier i tabellen NASK i perioden 2021-2024. Værdierne på variablen &quot;AN.11 Faste aktiver, nettobeholdning, primo året&quot; er altså præcis dobbelt så store som de skulle have været.',
  'mandatory

In [35]:
# Initialize the class with the target table
Kapital_landbrug = DstApi('NABK69')

# Get the table summary
Kapital_landbrug.tablesummary(language='da')

Table NABK69: Akkumulations- og statuskonti, investering og beholdning af faste aktiver (69-gruppering) efter beholdning / strøm, aktiv, branche, prisenhed og tid
Last update: 2025-07-04T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,BEHOLD,9,LSN,"AN.11 Faste aktiver, nettobeholdning, primo året",LEG,"Faste aktiver, bruttobeholdning, ultimo året",False
1,AKTIV,8,N11,Faste aktiver,N117,Intellektuelle rettigheder,False
2,BRANCHE,71,V,I alt,V97000,97000 Private husholdninger med ansat medhjælp,False
3,PRISENHED,2,V,Løbende priser,LAN,"2020-priser, kædede værdier",False
4,Tid,59,1966,1966,2024,2024,True


In [36]:
levels = Kapital_landbrug.variable_levels('BEHOLD', language='da')
print(levels.to_string(index=False))


  id                                             text
 LSN AN.11 Faste aktiver, nettobeholdning, primo året
 LSG      Faste aktiver, bruttobeholdning, primo året
P51G                  P.51g Faste bruttoinvesteringer
P51C                P.51c Forbrug af fast realkapital
  K3                          K.3 Tab ved katastrofer
  K7           K.7 Nominelle kapitalgevinster og -tab
 K72       heraf K.7.2 Reale kapitalgevinster og -tab
 LEN AN.11 Faste aktiver, nettobeholdning ultimo året
 LEG     Faste aktiver, bruttobeholdning, ultimo året


In [37]:
levels = Kapital_landbrug.variable_levels('BRANCHE', language='da')
print(levels.to_string(index=False))

    id                                                          text
     V                                                         I alt
 VMEMO                       Heraf: Offentlig forvaltning og service
V01000                                    01000 Landbrug og gartneri
V02000                                                02000 Skovbrug
V03000                                                 03000 Fiskeri
V06090                                        06090 Råstofindvinding
V10120                    10120 Føde-, drikke- og tobaksvareindustri
V13150                               13150 Tekstil- og læderindustri
V16000                                             16000 Træindustri
V17000                                           17000 Papirindustri
V18000                                          18000 Trykkerier mv.
V19000                                   19000 Olieraffinaderier mv.
V20000                                         20000 Kemisk industri
V21000                            

In [38]:
params = {
    'table': 'NABK69',
    'format': 'BULK',
    'variables': [
        {'code': 'BEHOLD', 'values': [
    "LEN", "P51G"
]},
        {'code': 'AKTIV', 'values': [
    "N11"
]},
        {'code': 'BRANCHE', 'values': [
    "V01000",  # 010000 Landbrug og gartneri
    "V10120",  # 10120 Føde-, drikke- og tobaksvareindustri
]},
        {'code': 'PRISENHED', 'values': [
    "LAN"
]},
        {'code': 'TID', 'values': ['*']}
    ]
}
r = requests.post('https://api.statbank.dk/v1' + '/data', json=params)
print(r.text[:200])

BEHOLD;AKTIV;BRANCHE;PRISENHED;TID;INDHOLD
AN.11 Faste aktiver, nettobeholdning ultimo året;Faste aktiver;01000 Landbrug og gartneri;2020-priser, kædede værdier;1966;222779
AN.11 Faste aktiver, nett


In [39]:
df = pd.read_csv(
    StringIO(
        requests.post('https://api.statbank.dk/v1' + '/data', json=params).text
    ), sep=';'
)

In [40]:
# 1) Sørg for at INDHOLD er tal (DK-format: komma som decimal)
df["INDHOLD"] = (
    df["INDHOLD"].astype(str)
      .str.replace("\u00a0", "", regex=False)  # non-breaking space
      .str.replace(" ", "", regex=False)       # mellemrum som tusindtals-sep
      .str.replace(".", "", regex=False)       # punktum som tusindtals-sep (hvis findes)
      .str.replace(",", ".", regex=False)      # komma -> punktum (decimal)
)
df["INDHOLD"] = pd.to_numeric(df["INDHOLD"], errors="coerce")

In [41]:
df.drop(columns=['PRISENHED'],inplace=True)
df.drop(columns=['AKTIV'],inplace=True)
df.to_csv('Data/Kapital_landbrugsdata.csv', index=False)
df

,BEHOLD,BRANCHE,TID,INDHOLD
0,"AN.11 Faste aktiver, nettobeholdning ultimo året",01000 Landbrug og gartneri,1966,222779.0
1,"AN.11 Faste aktiver, nettobeholdning ultimo året","10120 Føde-, drikke- og tobaksvareindustri",1966,34654.0
2,P.51g Faste bruttoinvesteringer,01000 Landbrug og gartneri,1966,NaN
3,P.51g Faste bruttoinvesteringer,"10120 Føde-, drikke- og tobaksvareindustri",1966,NaN
4,"AN.11 Faste aktiver, nettobeholdning ultimo året",01000 Landbrug og gartneri,1967,224430.0
...,...,...,...,...
227,P.51g Faste bruttoinvesteringer,"10120 Føde-, drikke- og tobaksvareindustri",2022,7507.0
228,"AN.11 Faste aktiver, nettobeholdning ultimo året",01000 Landbrug og gartneri,2023,187393.0
229,"AN.11 Faste aktiver, nettobeholdning ultimo året","10120 Føde-, drikke- og tobaksvareindustri",2023,103486.0
230,"AN.11 Faste aktiver, nettobeholdning ultimo året",01000 Landbrug og gartneri,2024,184590.0
